### Milestone 1 - Function Level Profiling

In [10]:
import cProfile,pstats
from mandelbort import compute_mandelbrot_naive, compute_mandelbrot_vectorized

In [12]:
cProfile.run('compute_mandelbrot_naive(-2,1,-1.5,1.5,1024)','Naive_Profile.prof')
cProfile.run('compute_mandelbrot_vectorized(-2,1,-1.5,1.5,1024)','Numpy_Profile.prof')

for name in ('Naive_Profile.prof','Numpy_Profile.prof'):    
    stats = pstats.Stats(name)
    stats.sort_stats('cumulative')
    stats.print_stats(10)

Thu Feb 26 13:58:06 2026    Naive_Profile.prof

         23008342 function calls in 10.267 seconds

   Ordered by: cumulative time
   List reduced from 20 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000   10.267   10.267 {built-in method builtins.exec}
        1    0.001    0.001   10.267   10.267 <string>:1(<module>)
        1    2.196    2.196   10.267   10.267 c:\Numerical_SC\mandelbrot-nsc\mandelbort.py:39(compute_mandelbrot_naive)
  1048576    6.806    0.000    8.070    0.000 c:\Numerical_SC\mandelbrot-nsc\mandelbort.py:59(mandelbrot_point_naive)
 21959734    1.264    0.000    1.264    0.000 {built-in method builtins.abs}
        2    0.000    0.000    0.000    0.000 c:\Users\rasm2\miniforge3\envs\nsc2026\Lib\site-packages\numpy\_core\function_base.py:26(linspace)
        2    0.000    0.000    0.000    0.000 {built-in method numpy.zeros}
        2    0.000    0.000    0.000    0.000 {built-in metho

##### Function which takes the most total time ?
Naive mandel point naive function has total time of 6.806 s

For numpy cant see any times just 0.00000 s
##### Are there any functions called suprising amount of times ? 
The absolut function is being called 21959734 times while the naive mandel point function is called 1048576 times

##### How Does the Numpy profile compare to naive ? 

It cannot find the functions/read it, as the timings for functions are all zero for the numpy

So the only thing to compare is the total time of the function of 1.001 sec vs 10.267 sec